In [7]:
import pandas as pd
import datetime

In [14]:
df_train = pd.read_csv("./data/predict-energy-behavior-of-prosumers/train.csv")
df_client=pd.read_csv("./data/predict-energy-behavior-of-prosumers/client.csv")
df_electricity=pd.read_csv("./data/predict-energy-behavior-of-prosumers/electricity_prices.csv")



FileNotFoundError: [Errno 2] No such file or directory: './train_files/train.csv'

In [10]:
df_train.sample(10000)

NameError: name 'df_train' is not defined

In [11]:
df_client

NameError: name 'df_client' is not defined

In [12]:
df_electricity

NameError: name 'df_electricity' is not defined

In [13]:
df_electricity["origin_date"].nunique()

NameError: name 'df_electricity' is not defined

In [ ]:
df_electricity["date"] = pd.to_datetime(df_electricity["forecast_date"])
df_electricity["date"] = df_electricity["date"].apply(lambda x : x.strftime("%d-%m-%y"))
df_electricity["date"]

In [ ]:
df_client["date_str"] = df_client["date"].apply(lambda x : str(x))
df_electricity["date_str"] = df_electricity["date"].apply(lambda x : str(x))

## Feature Engineering

In [ ]:
df_electricity

## Creation colonne

In [ ]:
df_train["price_elec"] = -1

for unique_date in df_train["datetime"].unique()[:10]:
  # Récuperer le prix de l'electricite a ce jour la
  df_elec_filtered = df_electricity[df_electricity["origin_date"] == unique_date]
  if len(df_elec_filtered) != 0:
    if len(df_elec_filtered) == 1:
      elec_price = df_elec_filtered["euros_per_mwh"]
    else : 
      elec_price = df_elec_filtered.iloc[0]["euros_per_mwh"]

    # Pour toutes les lignes de df_train a ce jour la, insérer le prix
    df_train_filtered = df_train[df_train["datetime"] == unique_date]
    df_train_filtered["price_elec"] = elec_price

In [ ]:
df_train.describe()

In [ ]:
def find_time(group):
  df_electricity[df_electricity["origin_date"] == group.iloc[0]["datetime"]].iloc[0]["euros_per_mwh"]
  

df_train.groupby("datetime")["electricity price"] = df_train.groupby("datetime").apply(find_time)

In [ ]:
import time

def find_electricity_price(line):
  if int(line.name) % 2000 == 0 : 
    print(line.name, "/", df_train.shape[0],
          (line.name/df_train.shape[0])*100, "%", end="\r")
    time.sleep(2)
  return df_electricity[df_electricity["origin_date"] == line["datetime"]].iloc[0]["euros_per_mwh"]

df_train["euros_per_mwh"] = df_train.apply(find_electricity_price, axis = 1)
df_train

In [ ]:
# df_train["euros_per_mwh"] = 
df_train.apply(lambda x : df_electricity[df_electricity["origin_date"] == x["datetime"]].values, axis = 1)